In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import copy


In [ ]:
import numpy as np

In [ ]:
import argparse
import wandb
import yaml
import random
import numpy as np

from training import train, evaluate
from config import Config
from getters import *
from models import *

from utils import find_file


In [ ]:


def make_model(name, **kwargs):
    Model = get_model(name)
    return Model(**kwargs)

In [ ]:
expe_name = 'copy_hadamssm_2'
task2sweep = {'copy_hadamssm': 'test',
              'copy_bjorck': 'sweep_ucvrt4w4',
              'copy_projunn': 'sweep_a4d2sihj',
             'pmnist_bjorck': 'sweep_bgckd728',
             'smnist': 'sweep_a3af3t6v',
             'ptb_bjorck': 'sweep_g0tjzjy7',
}

short2fullname = {
    'copy_hadamssm_2': 'hadamSSM_copy',
    'copy_bjorck_5' : 'fresh-sweep-19_qeh5ezkj',
    'copy_bjorck_6' : 'rich-sweep-25_eihml4ya',
    'copy_bjorck_8' : 'woven-sweep-33_dfdq24vp',
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 'rose-sweep-22_e3ftelec',
    'copy_projunn_8' : 'resilient-sweep-32_fze6523n',
    'pmnist_bjorck_3' : 'rare-sweep-1_uyv83nl9',
    'pmnist_bjorck_4' : 'rare4-sweep-1_uyv83nl9',
    'pmnist_bjorck_5' : 'fresh-sweep-2_c9nwwvj4',
    'pmnist_bjorck_6' : 'skilled-sweep-3_1pi0p71d',
    'pmnist_bjorck_8' : 'denim-sweep-4_27qjq5oa',
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 'sleek-sweep-2_275j9jq7',
    'smnist_bjorck_5' : 'wandering-sweep-3_1tbbz1et',
    'smnist_bjorck_6' : 'distinctive-sweep-4_c0gyg62r',
    'smnist_bjorck_8' : 'rich-sweep-5_ypocgzgv',
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 'earnest-sweep-2_rb6cg3gh',
    'ptb_bjorck_5' : 'eternal-sweep-3_5vss2or6',
    'ptb_bjorck_6' : 'polished-sweep-4_qa3yj567',
    'ptb_bjorck_8' : 'eager-sweep-5_whv7y4xu',
}

In [ ]:
def get_task_value(expe_name,task2value):
    value = None
    for kk in task2value.keys():
        if kk in expe_name:
            value = task2value[kk]
            break
    return value

In [ ]:
path2yaml = {'copy': 'copy_task',
             'pmnist': 'permuted_mnist',
             'smnist': 'seq_mnist',
             'ptb': 'ptb',}

config_path = get_task_value(expe_name,path2yaml)
print(config_path)

sweep_name = get_task_value(expe_name,task2sweep)

In [ ]:
sweep_name

In [ ]:
config_file_path = "config/"+config_path+"/"+short2fullname[expe_name]+".yaml"
weight_file_path = "results/"+sweep_name+"_"+short2fullname[expe_name]+".pth"
result_file_path = "results/"+sweep_name+"_"+short2fullname[expe_name]+".txt"

In [ ]:
config_file_path = "config/"+config_path+"/"+"copy_task_hadamSSM_config_test.yaml"


In [ ]:
device = 'cpu' #'cpu' #'cpu' #'cuda:0'

In [ ]:
config_file_path


## Verification of model load an performance with FP activations

In [ ]:

conf_file = find_file(config_file_path)
config = Config(conf_file=conf_file)

In [ ]:
config.model['num_bits_feat'] = None 

In [ ]:

model = make_model(**config.model).to(device)
print(model)


In [ ]:
model.load_state_dict(torch.load(weight_file_path))

In [ ]:
print(config.dataset)

In [ ]:
Dataset = get_dataset(config.dataset['name'])

dataset = Dataset(**config.dataset)


In [ ]:
device

In [ ]:

if dataset.test_ds is not None:
    test_batch_size = dataset.te_size // 10
    test_ds = dataset.test_ds
else:
    test_batch_size = dataset.va_size // 10
    test_ds = dataset.val_ds
stat_test = evaluate(test_ds, test_batch_size, model, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    

In [ ]:
print(stat_test)
#ref  {'test_loss': 9.112985571846366e-05}   #in cuda {'test_loss': 0.00011134522355860099} ?????
# open and print result_file_path
with open(result_file_path, 'r') as f:
    print(f.read())


## Prepare PTQ of activations:
Compute max value of activation on training set

TO BE DONE ONCE ONLY => CELLS IN MARKDOWN

In [ ]:
weights = torch.load(weight_file_path)
new_weights = {}
new_weights["input_layer.weight"] = model.input_layer.qweight  #NO_PARAM model.input_layer.weight
new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight
if 'activation.b' in weights:
    new_weights['activation.b'] = weights['activation.b']
if 'recurrent_layer.bias' in weights:
    new_weights['recurrent_layer.bias'] = weights['recurrent_layer.bias']
#new_weights['output_layer.weight'] = weights['output_layer.weight']

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

In [ ]:
print(list(torch.unique(new_weights["input_layer.weight"]).numpy()))
print(list(torch.unique(new_weights["recurrent_layer.weight"]).detach().numpy()))

In [ ]:
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QSSM'
#config_scaled.model['bias'] = False
if 'activation.b' in new_weights:
    config_scaled.model['activation'] = layers.modrelu(**config_scaled.model['activation_config'])
else:
    config_scaled.model['activation'] = torch.nn.ReLU()
config_scaled.model['num_bits_feat'] = 16

In [ ]:
device_stat = 'cuda'
model_stat = make_model(**config_scaled.model).to(device_stat)
model_stat.load_state_dict(new_weights)

In [ ]:
model_stat

In [ ]:
#compute max_val on training set
train_batch_size = dataset.tr_size // 1000
stat_test = evaluate(dataset.train_ds, train_batch_size, model_stat, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device_stat)


In [ ]:
print("stats on training set")
print(stat_test)
print("input")
print(model_stat.quant_input.stat_max_absolute)
print("feat")
print(model_stat.quant_feat.stat_max_absolute)

#stats on training set
#input
#1.0
#feat
#19.80474853515625

In [ ]:
#stat_test = evaluate(test_ds, test_batch_size, model_stat, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device_stat)
#print(stat_test)

In [ ]:
expe_name

In [ ]:

task2input_max_absolute = {'copy': 2.0,
             'pmnist': 1.0,
             'smnist': 1.0,
             'ptb': 2.0,}

short2feat_max_absolute  = {
    'copy_hadamssm_2': 170.156494140625, #21.454788208007812,
    'copy_bjorck_5' : 24.19596290588379,
    'copy_bjorck_6' : 42.483619689941406,
    'copy_bjorck_8' : 30.833498001098633, #65.3037109375,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 1902.8502197265625,
    'copy_projunn_8' : 219.551513671875,
    'pmnist_bjorck_3' : 22.018991470336914,
    'pmnist_bjorck_4' : 26.795150756835938,
    'pmnist_bjorck_5' : 25.203166961669922,
    'pmnist_bjorck_6' : 25.342670440673828,
    'pmnist_bjorck_8' : 26.046796798706055,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 17.66891860961914,
    'smnist_bjorck_5' : 29.19618034362793,
    'smnist_bjorck_6' : 30.9416561126709,
    'smnist_bjorck_8' : 30.775388717651367,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 30.078453063964844,
    'ptb_bjorck_5' : 6.935123443603516,
    'ptb_bjorck_6' : 6.048137187957764,
    'ptb_bjorck_8' : 9.994182586669922,
}

task2prescale = {'copy': 1.0,
             'pmnist': 255.0/256.0,
             'smnist': 255.0/256.0,
             'ptb': 1.0,}

task2shiftinput = {'copy': 0,
             'pmnist': 8,
             'smnist': 8,
             'ptb':0,}

In [ ]:

saved_quant_input_max_absolute = get_task_value(expe_name,task2input_max_absolute) #short2input_max_absolute[expe_name] #2. # 2.  ## set it twice to get 1 as max
saved_quant_feat_max_absolute = short2feat_max_absolute[expe_name] #19.80474853515625   #32 #

input_pre_scaling_factor = get_task_value(expe_name,task2prescale)
#input_pre_scaling_factor = 1.0 # to get true Int
#input_pre_scaling_factor = 255.0/256.0
shift_input = get_task_value(expe_name,task2shiftinput)
#shift_input = 0
#shift_input = 8


In [ ]:
print(saved_quant_input_max_absolute)
print(saved_quant_feat_max_absolute)
print(input_pre_scaling_factor)
print(shift_input)

## Get $\alpha_W$ and $\alpha_U$ values

TO BE DONE ONCE ONLY => CELLS IN MARKDOWN

In [ ]:
model.recurrent_layer.parametrizations.weight

In [ ]:
'''model.recurrent_layer.parametrizations.weight.original
aa = model.recurrent_layer.parametrizations.weight[0](model.recurrent_layer.parametrizations.weight.original)
model.recurrent_layer.parametrizations.weight[-2](aa)'''

In [ ]:
def computeParametrization(params,originaW,until_step=2):
    for kk in range(until_step):
        originaW = params[kk](originaW)
    return originaW

In [ ]:
#Get floating point weights
import torch.nn.utils.parametrize as parametrize

print(model.recurrent_layer.parametrizations.weight)
if len(model.recurrent_layer.parametrizations.weight) > 1: #1 is only Quantize, SpectralLinear has 3 paramtrizations
    Wfp = computeParametrization(model.recurrent_layer.parametrizations.weight,
                                 model.recurrent_layer.parametrizations.weight.original,
                                 until_step=2) #model.recurrent_layer.parametrizations.weight[-2](model.recurrent_layer.parametrizations.weight.original)
else:
    Wfp = model.recurrent_layer.weight
print("W",Wfp.min().detach().numpy(),Wfp.max().detach().numpy(),np.max(np.abs(Wfp.detach().numpy())))
if parametrize.is_parametrized(model.input_layer):
    print(model.input_layer.parametrizations.weight)
    if len(model.input_layer.parametrizations.weight) > 1:
        Ufp = computeParametrization(model.input_layer.parametrizations.weight,
                                    model.input_layer.parametrizations.weight.original,
                                    until_step=2) #model.input_layer.parametrizations.weight[-2](model.input_layer.parametrizations.weight.original)
    else:
        Ufp = model.input_layer.parametrizations.weight.original
else:
    Ufp = model.input_layer.qweight #QLinear implementation of Armand 
print("U",Ufp.min().detach().numpy(),Ufp.max().detach().numpy(),np.max(np.abs(Ufp.detach().numpy())))

In [ ]:

short2alpha_w  = {
    'copy_hadamssm_2': 0.25,
    'copy_bjorck_5' : 0.26512054,
    'copy_bjorck_6' : 0.28184956,
    'copy_bjorck_8' : 0.26092893, #0.26838166,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 0.9988483,
    'copy_projunn_8' : 0.9998472,
    'pmnist_bjorck_3' : 0.32128334,
    'pmnist_bjorck_4' : 0.3660844,
    'pmnist_bjorck_5' : 0.34444174,
    'pmnist_bjorck_6' : 0.38659847,
    'pmnist_bjorck_8' : 0.6006793,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 0.43379927,
    'smnist_bjorck_5' : 0.36560023,
    'smnist_bjorck_6' : 0.40944064,
    'smnist_bjorck_8' : 0.40728217,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 0.19523722,
    'ptb_bjorck_5' : 0.23500842,
    'ptb_bjorck_6' : 0.16609415,
    'ptb_bjorck_8' : 0.48273855,
}

short2alpha_u  = {
    'copy_hadamssm_2': 5.5423894,
    'copy_bjorck_5' : 2.2110918,
    'copy_bjorck_6' : 2.2872915,
    'copy_bjorck_8' : 1.733867, #1.9520738,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 7.8092656,
    'copy_projunn_8' : 1.9943593,
    'pmnist_bjorck_3' : 16.959444,
    'pmnist_bjorck_4' : 12.96517,
    'pmnist_bjorck_5' : 8.487175,
    'pmnist_bjorck_6' : 10.693445,
    'pmnist_bjorck_8' : 7.431476,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 6.031232,
    'smnist_bjorck_5' : 8.092017,
    'smnist_bjorck_6' : 5.021599,
    'smnist_bjorck_8' : 6.9251447,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 15.941487,
    'ptb_bjorck_5' : 2.4994867,
    'ptb_bjorck_6' : 2.4208195,
    'ptb_bjorck_8' : 7.541183,
}

In [ ]:
saved_alpha_u = short2alpha_u[expe_name] #np.max(np.abs([-0.6072238087654114, 0.607637882232666]))
saved_alpha_w = short2alpha_w[expe_name] #nnp.max(np.abs([-0.24883395433425903, 0.24531713128089905]))

In [ ]:
print(saved_alpha_u,saved_alpha_w)

## Rounding functions

In [ ]:
def round_values(vv, num_bits):
    vv = vv*2**num_bits
    assert torch.max(torch.abs(vv.round() - vv))<1e-4   # ?????
    vv = vv.round()
    vv = vv/2**num_bits
    return vv

def verify_round_max(vv, num_bits):
    eval_numbits = 0
    for ss in range(num_bits):
        vv = vv*2
        if torch.max(torch.abs(vv.round() - vv))<1e-5:
            eval_numbits = ss+1
            break
    assert eval_numbits > 0
    print(eval_numbits,"max",num_bits)
    return eval_numbits
    
  
def verify_and_round(vv, num_bits):  
    eval_numbits = verify_round_max(vv, num_bits)
    return round_values(vv, eval_numbits)


## Fully Quantized RNN

In [ ]:
weights = torch.load(weight_file_path)

In [ ]:
vv = (2**config.model['num_bits'])*model.recurrent_layer.weight/ saved_alpha_w

In [ ]:
torch.max(torch.abs(vv.round()-vv))

In [ ]:
len(torch.unique(model.input_layer.qweight))

In [ ]:
new_weights = {}
requant_input_weight = round_values(model.input_layer.qweight/ saved_alpha_u, config.model['num_bits'])#NO_PARAM model.input_layer.weight
new_weights["input_layer.weight"] = requant_input_weight #model.input_layer.weight/ (saved_quant_input_max_absolute*saved_alpha_u)
float_recurrent_weight = model.recurrent_layer.weight
requant_recurrent_weight = round_values(model.recurrent_layer.weight/ saved_alpha_w, config.model['num_bits'])
new_weights['recurrent_layer.weight'] = requant_recurrent_weight #model.recurrent_layer.weight / saved_alpha_w ## compensate alpha_w in hidden quantization
#new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight  ## compensate alpha_w in hidden quantization
scaled_quant_feat_max_absolute  = saved_quant_feat_max_absolute/saved_alpha_u
print(scaled_quant_feat_max_absolute) #saved_quant_feat_max_absolute*saved_alpha_w/(saved_quant_input_max_absolute*saved_alpha_u))
print("scaled hidden quantiz",scaled_quant_feat_max_absolute*saved_alpha_w)  #4.055123978191649
#new_quant_feat_max_absolute  = scaled_quant_feat_max_absolute #8/saved_alpha_w #saved_alpha_w*saved_quant_feat_max_absolute/(saved_quant_input_max_absolute*saved_alpha_u)
if 'activation.b' in weights:
    new_weights['activation.b'] = weights['activation.b']/ saved_alpha_u
    print("activation.b max abs",torch.abs(new_weights['activation.b']).max().cpu().detach().numpy())
if 'recurrent_layer.bias' in weights:
    new_weights['recurrent_layer.bias'] = weights['recurrent_layer.bias']/ saved_alpha_u
    print("'recurrent_layer.bias' max abs",torch.abs(new_weights['recurrent_layer.bias']).max().cpu().detach().numpy())
new_weights['output_layer.weight'] = saved_alpha_u*weights['output_layer.weight']/input_pre_scaling_factor

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

In [ ]:
print(saved_quant_feat_max_absolute,saved_alpha_u,saved_quant_feat_max_absolute/saved_alpha_u)
print(scaled_quant_feat_max_absolute,saved_alpha_w,scaled_quant_feat_max_absolute*saved_alpha_w)

In [ ]:

short2scaled_feat_max_absolute  = {  # if possible power of two
    'copy_hadamssm_2': 8.0,
    'copy_bjorck_5' : 4.0,
    'copy_bjorck_6' : 4.0,  
    'copy_bjorck_8' : 4.0, #12.0, #4.417504669503896
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 256.0,
    'copy_projunn_8' : 128.0,
    'pmnist_bjorck_3' : 1.0,    #0.20856624553910355
    'pmnist_bjorck_4' : 1.0,    #0.7565875871836489
    'pmnist_bjorck_5' : 1.0,
    'pmnist_bjorck_6' : 1.0,
    'pmnist_bjorck_8' : 2.0,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 2.0,
    'smnist_bjorck_5' : 2.0,
    'smnist_bjorck_6' : 4.0,
    'smnist_bjorck_8' : 2.0,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 1.0,
    'ptb_bjorck_5' : 1.0,
    'ptb_bjorck_6' : 1.0,
    'ptb_bjorck_8' : 1.0,
}

In [ ]:
new_quant_feat_max_absolute  = short2scaled_feat_max_absolute[expe_name]  #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)

new_bias_max_absolute = 1.0

### check rounding of weights 

In [ ]:
iww = 2**(config.model['num_bits']-1)*2*new_weights["input_layer.weight"].unique()
print("input ",torch.norm(iww.round() - iww))

iww = 2**(config.model['num_bits']-1)*requant_recurrent_weight
#print(iww)
print("recurrent ",torch.norm(iww.round() - iww))

### Evaluate with several bitwidth for activation

In [ ]:
print(config.model)

In [ ]:
import copy
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QSSM'
#config_scaled.model['bias'] = False
if 'activation.b' in new_weights:
    config_scaled.model['activation'] = layers.modified4q_modrelu(**config_scaled.model['activation_config'])
else:
    config_scaled.model['activation'] = torch.nn.ReLU()

In [ ]:
from extra_layers import _Quantize_stats

qb = None
models = []
for qb in [None,12]: #6,8,10,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        new_weights['recurrent_layer.weight'] = float_recurrent_weight
        #model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        new_weights['recurrent_layer.weight'] = requant_recurrent_weight
    model_scaled.load_state_dict(new_weights)
    if qb is not None:      
        model_scaled.quant_input.pre_scaling_factor = input_pre_scaling_factor     
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute #saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w     
        #print(model_scaled.activation.b)
        #print(quant_bias(model_scaled.activation.b))
        if ('activation.b' in new_weights) or ('recurrent_layer.bias' in new_weights):
            #quantize biases
            quant_bias = _Quantize_stats(weight=None,num_bits= model_scaled.quant_feat.num_bits, max_absolute= new_bias_max_absolute)
            if 'activation.b' in new_weights:
                model_scaled.activation.b.data = quant_bias(model_scaled.activation.b) 
            if 'recurrent_layer.bias' in new_weights:
                model_scaled.recurrent_layer.bias.data = quant_bias(model_scaled.recurrent_layer.bias)
    models.append(model_scaled)

In [ ]:
x,y = next(iter(test_ds))
x0 = x[None,:] #No batch_size [0:1]
x0.shape

In [ ]:

model_float = models[0]
model_float.eval()
model_float(x0.to(device))

print("input")
#print(model_float.quant_input.stat_max_absolute)
print("feat")
print(model_float.quant_feat.stat_max_absolute)

In [ ]:
torch.unique(model_float.input_layer.weight)

In [ ]:
model_scaled.quant_feat.__dict__

In [ ]:
h0 = hidden_scaled[-1][0,220:230]
h1 = h0*model_scaled.quant_feat.pre_scaling_factor
h2 = h1/model_scaled.quant_feat.max_absolute
h3 = h2*2**(model_scaled.quant_feat.num_bits-1)
h4 = h3.round()
h5 = h4/2**(model_scaled.quant_feat.num_bits-1)
h6 = h5*model_scaled.quant_feat.max_absolute
h7 = h6/model_scaled.quant_feat.pre_scaling_factor
print(h0,"\n",h1,"\n",h2,"\n",h3,"\n",h4,"\n",h5,"\n",h6,"\n",h7)
print(torch.abs(h0-h7))
print(model_scaled.quant_feat(hidden_scaled[-1][0,220:230])/model_scaled.quant_feat.pre_scaling_factor)

In [ ]:
import torch.nn.functional as F
model_float = models[0]
model_scaled = models[1]
hidden = []
hidden_scaled = []
hidden_scaled_input = []
hidden_scaled_quant = []
hidden_scaled_rec = []
for ii in range(1000):
    print("input",x0[0:1,ii])
    print("qinput",model_scaled.quant_input(x0[0:1,ii])/input_pre_scaling_factor)
    #print(model.input_layer(x0[0:1,ii]))
    #print(model_scaled.input_layer(x0[0:1,ii])*saved_quant_input_max_absolute*saved_alpha_u)
    print("diff U.x",torch.norm(model_float.input_layer(x0[0:1,ii])-model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))/input_pre_scaling_factor))
    
    if len(hidden)==0:
        hidden.append(model_float.input_layer(x0[0:1,ii]))
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
        #print(hidden_scaled[0])
        #print(hidden_scaled[0]*2**(config.model['num_bits']))
        hidden_scaled_quant.append(torch.zeros_like(hidden_scaled[0])) # fake
        hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])) # fake
    else:
        #print("hidden",hidden[-1][0,0:10])
        #print("hidden_rec",model.recurrent_layer(hidden[-1])[0,0:10])
        hidden.append(model_float.input_layer(x0[0:1,ii])+model_float.recurrent_layer(hidden[-1]))
        #print("hidden_scaled",hidden_scaled[-1][0,0:10])
        #print("hidden_scaled",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled_quant.append(model_scaled.quant_feat(hidden_scaled[-1]))
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10])
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))
        print("diff_hidden_scaled",torch.norm(hidden[-2]-hidden_scaled[-1]/input_pre_scaling_factor))
        print("diff_hidden_quant",torch.norm(hidden[-2]-hidden_scaled_quant[-1]/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor)))
        diff_hidden_scaled = (hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor-hidden_scaled[-1])*saved_alpha_u/input_pre_scaling_factor
        print("quant diff hidden",torch.norm(diff_hidden_scaled).detach().numpy(),torch.min(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.mean(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.max(torch.abs(diff_hidden_scaled)).detach().numpy(),diff_hidden_scaled.shape)
        #print("hidden_scaled_reccccc",model.recurrent_layer(hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))[0,0:10])
        #print("hidden_scaled_reccccc1",model.recurrent_layer(hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor)[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(hidden_scaled_quant[-1])[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1])))
        hidden_scaled_rec.append(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))) 
    
    hidden_scaled_input.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
    #print("hidden_scaled_rec",hidden_scaled_rec[-1]*saved_alpha_u/input_pre_scaling_factor)
    print("diff h_t",torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor))
    print("h_t, min,mean,max,norm",hidden[-1].min().detach().numpy(),hidden[-1].mean().detach().numpy(),hidden[-1].max().detach().numpy(),torch.norm(hidden[-1]).detach().numpy())
    #print("diff h_t",hidden[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
    #if ii>2:
    #    aaa
    if 'activation.b' in new_weights:
        print("diff h_t+bias",torch.norm(hidden[-1]+model.activation.b-(hidden_scaled[-1]+model_scaled.activation.b)*saved_alpha_u/input_pre_scaling_factor))
    #NO ACTIV IN SSM hidden[-1] = model.activation(hidden[-1])
    #NO ACTIV IN SSM hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print("activ",torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor),torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor)/torch.norm(hidden[-1]),torch.max(torch.abs(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor)))
    #print("activ value",hidden[-1],hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)
    #print("activ valueq",hidden_scaled[-1],model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor)
    print("activ quantif",torch.norm(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor))
    '''if torch.max(torch.abs(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor))>0.1:
        print(hidden[-1][0,290:300])
        print(hidden_scaled[-1][0,290:300]/input_pre_scaling_factor)
        print(model_scaled.quant_feat(hidden_scaled[-1])[0,290:300]/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor)
        diff = hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor
        print(torch.argmax(torch.abs(diff)))
        print("diff",diff[0,290:300],diff.min().detach().numpy(),diff.mean().detach().numpy(),diff.max().detach().numpy(),torch.norm(diff).detach().numpy())
        break'''

In [ ]:
model_scaled.quant_feat.__dict__


In [ ]:
from extra_layers import _Quantize_stats
results = {}
for qb in [None,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        new_weights['recurrent_layer.weight'] = float_recurrent_weight
        #model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        new_weights['recurrent_layer.weight'] = requant_recurrent_weight
    model_scaled.load_state_dict(new_weights)
    if qb is not None:      
        model_scaled.quant_input.pre_scaling_factor = input_pre_scaling_factor     
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute #saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w     
        #print(model_scaled.activation.b)
        #print(quant_bias(model_scaled.activation.b))
        if ('activation.b' in new_weights) or ('recurrent_layer.bias' in new_weights):
            #quantize biases
            quant_bias = _Quantize_stats(weight=None,num_bits= model_scaled.quant_feat.num_bits, max_absolute= new_bias_max_absolute)
            if 'activation.b' in new_weights:
                model_scaled.activation.b.data = quant_bias(model_scaled.activation.b) 
            if 'recurrent_layer.bias' in new_weights:
                model_scaled.recurrent_layer.bias.data = quant_bias(model_scaled.recurrent_layer.bias)
    stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] =stat_test.copy()

for q in results:
    print(q,results[q])
#save perf in txt file
with open("results/"+expe_name+"_"+sweep_name+"_"+short2fullname[expe_name]+"_quant.txt", "w") as f:
    for q in results:
        for kk in results[q]:
            f.write(str(q)+";"+str(kk)+";"+str(results[q][kk])+"\n")



## Verification of the intermediate computation: chek for quantization

need to simulate the QRNN computation and save intermediate values 

In [ ]:
model_scaled.quant_feat.num_bits

In [ ]:
x,y = next(iter(test_ds))
x0 = x[None,:] #No batch_size [0:1]
x0.shape

In [ ]:
if 'activation.b' in new_weights:
    model.activation.b/ saved_alpha_u

In [ ]:

if 'activation.b' in new_weights:
    model_scaled.activation.b

In [ ]:
x0.shape

In [ ]:
saved_quant_input_max_absolute
input_pre_scaling_factor

In [ ]:

import torch.nn.functional as F
hidden = []
hidden_scaled = []
hidden_scaled_input = []
hidden_scaled_quant = []
hidden_scaled_rec = []
for ii in range(1000):
    print("input",x0[0:1,ii])
    print("qinput",model_scaled.quant_input(x0[0:1,ii])/input_pre_scaling_factor)
    #print(model.input_layer(x0[0:1,ii]))
    #print(model_scaled.input_layer(x0[0:1,ii])*saved_quant_input_max_absolute*saved_alpha_u)
    print("diff U.x",torch.norm(model.input_layer(x0[0:1,ii])-model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))*saved_alpha_u/input_pre_scaling_factor))
    
    if len(hidden)==0:
        if 'recurrent_layer.bias' in new_weights:
                hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer.bias)
                hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer.bias)
                hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])+model_scaled.recurrent_layer.bias) # fake
        else:
            hidden.append(model.input_layer(x0[0:1,ii]))
            hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
            hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])) # fake
        #print(hidden_scaled[0])
        #print(hidden_scaled[0]*2**(config.model['num_bits']))
        hidden_scaled_quant.append(torch.zeros_like(hidden_scaled[0])) # fake
    else:
        #print("hidden",hidden[-1][0,0:10])
        #print("hidden_rec",model.recurrent_layer(hidden[-1])[0,0:10])
        hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer(hidden[-1]))
        #print("hidden_scaled",hidden_scaled[-1][0,0:10])
        #print("hidden_scaled",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled_quant.append(model_scaled.quant_feat(hidden_scaled[-1]))
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10])
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))
        print("diff_hidden_scaled",torch.norm(hidden[-2]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor))
        print("diff_hidden_quant",torch.norm(hidden[-2]-hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor)))
        diff_hidden_scaled = (hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor-hidden_scaled[-1])*saved_alpha_u/input_pre_scaling_factor
        print("quant diff hidden",torch.norm(diff_hidden_scaled).detach().numpy(),torch.min(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.mean(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.max(torch.abs(diff_hidden_scaled)).detach().numpy(),diff_hidden_scaled.shape)
        #print("hidden_scaled_reccccc",model.recurrent_layer(hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))[0,0:10])
        #print("hidden_scaled_reccccc1",model.recurrent_layer(hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor)[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(hidden_scaled_quant[-1])[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1])))
        hidden_scaled_rec.append(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))) 
    
    hidden_scaled_input.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
    #print("hidden_scaled_rec",hidden_scaled_rec[-1]*saved_alpha_u/input_pre_scaling_factor)
    print("diff h_t",torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor))
    print("h_t, min,mean,max,norm",hidden[-1].min().detach().numpy(),hidden[-1].mean().detach().numpy(),hidden[-1].max().detach().numpy(),torch.norm(hidden[-1]).detach().numpy())
    #print("diff h_t",hidden[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
    #if ii>2:
    #    aaa
    if 'activation.b' in new_weights:
        print("diff h_t+bias",torch.norm(hidden[-1]+model.activation.b-(hidden_scaled[-1]+model_scaled.activation.b)*saved_alpha_u/input_pre_scaling_factor))
    #NO ACTIV IN SSM hidden[-1] = model.activation(hidden[-1])
    #NO ACTIV IN SSM hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print("activ",torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor),torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)/torch.norm(hidden[-1]),torch.max(torch.abs(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)))
    #print("activ value",hidden[-1],hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)
    #print("activ valueq",hidden_scaled[-1],model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor)
    print("activ quantif",torch.norm(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor*saved_alpha_u/input_pre_scaling_factor))
    

In [ ]:
print(model_scaled)

In [ ]:

preds = []
if config.model['manytomany']:
    for ii,hh in enumerate(hidden_scaled): #zip(hidden_scaled):
        #print(hh.shape)
        #print(hh[:,0:10])
        preds.append(model_scaled.output_layer(model_scaled.activation(hh)))
    nsmples = 10
else:
    preds.append(model_scaled.output_layer(model_scaled.activation(hidden_scaled[-1])))
    nsmples = 1
    '''if ii==2:
        break'''
    #preds.append(model_scaled.output_layer(hh[0]))
ypred = model_scaled(x0.to(device))
preds = torch.concat(preds,0)
print(ypred[-1:])    
print("preds",preds[-nsmples:])
print(torch.argmax(ypred[-nsmples:],dim=-1))    
print("preds",torch.argmax(preds[-nsmples:],dim=-1))
print(ypred.shape,preds.shape)

In [ ]:
vvin = verify_and_round(model_scaled.recurrent_layer.weight, config.model['num_bits'])

for hh, hhin, hhq, hhrec in zip(hidden_scaled,hidden_scaled_input, hidden_scaled_quant,hidden_scaled_rec):
    vvin = hhin*(2**(config.model['num_bits']-1+shift_input)) #+model_scaled.quant_input.num_bits-1)) 
    print("hhin ",torch.norm(vvin.round() - vvin), torch.min(vvin.round() - vvin), torch.max(vvin.round() - vvin))
    vvin = verify_and_round(hhin, config.model['num_bits']-1+shift_input)
    vvq = hhq*(2**(model_scaled.quant_input.num_bits-1)) 
    #vvq = hhq*(2**(shift_input+model_scaled.quant_input.num_bits-1)*model_scaled.quant_input.max_absolute) 
    print("hhq ",torch.norm(vvq.round() - vvq), torch.min(vvq.round() - vvq), torch.max(vvq.round() - vvq))
    vvq = verify_and_round(hhq, model_scaled.quant_feat.num_bits-1)  # +1 for multiply by 2 in model_scaled.quant_input.max_absolute
    vvrec = hhrec*(2**(config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)) 
    print("hhrec ",torch.norm(vvrec.round() - vvrec), torch.min(vvrec.round() - vvrec), torch.max(vvrec.round() - vvrec))
    vvrec = verify_and_round(hhrec, config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)
    vv = hh*(2**(config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)) 
    print("hh ",torch.norm(vv.round() - vv), torch.min(vv.round() - vv), torch.max(vv.round() - vv))
    vv = verify_and_round(hh, config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)
    
    

# Old stuf 

$U = \alpha_U.U_k$, $U_k\in[[-2^{k-1},2^{k-1}-1]]$

$W = \alpha_W.W_k$, $W_k\in[[-2^{k-1},2^{k-1}-1]]$

$x = x_{ka}/2^{ka-2}$  , $x_{ka}\in{0,2^{ka-2}}$ #$[-2^{ka-1},2^{ka-1}-1]]$

$h = \alpha_h.h_{ka}$, $h_{ka}\in[[-2^{ka-1},2^{ka-1}-1]]$

$h_{t+1} = W.h + U.x = (\alpha_h*\alpha_W)*(W_k.h_{ka}) + (\alpha_U/2^{ka-2})*(U_k*x_{ka})$

$h_{t+1} = ReLU(h_{t+1})$

On peut eviter la multiplaction par $\alpha_U$

Simplification si 

$1/\alpha_h*\alpha_W = 2^k'$


# Reminder fixed point notation  $Q_{l,k}$

$Q_{l,k} = {p/2^{k},p\in[[-2^{l-1},2^{l-1}-1]]}\in [-2^{l-k-1},2^{l-k-1}[$ with $l$ bits

in particular : $Q_k = Q_{k+1,k} = {p/2^{k},p\in[[-2^k,2^k-1]]}\in [-1,1[$ with $k+1$ bits

multiplication $Q_{l,k}.Q_{l',k'} -> Q_{l+l'-1,k+k'}$

$Q_{k}.Q_{k'} -> Q_{k+k'+1,k+k'}=Q_{k+k'}$

Addition (has to be with the same $k$)

$Q_{k} + Q_{k} -> Q_{k+2,k}$




# Formulation with fixed point variables
## matrices and vectors
$U = \alpha_U/2^{k-1}.U_k= \alpha_U.U'_k$, with $U'_k\in Q_{k-1}$

$W = \alpha_W/2^{k-1}.W_k= \alpha_W.W'_k$, with $W'_k\in Q_{k-1}$

$x(t) = 2*x_{ka}(t)$, with $x\in\{0,1\}$ , $x_{ka}\in\{0,0.5\}$ could be $Q_1$ (2 bits), but easier to keep it in $Q_{ka-1}$

$h(t) = \alpha_h/2^{ka-1}.h_{ka}(t)=\alpha_h.h'_{ka}(t)$ with $h'_{ka}(t)\in Q_{ka-1}$

## recurrent computation

$h(t+1) = W.h(t) + U.x(t) = (\alpha_h*\alpha_W)*(W'_k.h'_{ka}(t)) + (2\alpha_U)*(U'_k*x_{ka}(t))$ with $W'_k.h'_{ka}(t)$ and $U'_k*x_{ka}(t)\in Q_{k+ka-2}$

$h(t+1) = ReLU(h(t+1))$

or $h(t+1) = ModReLU(h(t+1),b) = sign(h(t+1))*ReLU(|h(t+1)|+b)$ => Could be quantized back to $\alpha_h.h'_{ka}(t+1)$



# Scaling hidden vector to reduce floating point multiplications ($2\alpha_U$)

To avoid the multiplication by  par $2\alpha_U$

scaled h : $sh(t) = h(t)/(2\alpha_U)$, $sh(t) = \alpha_{sh}.sh'_{ka}(t)$ with $\alpha_{sh} = \alpha_h/(2\alpha_U)$, and $sh'_{ka}(t)=h'_{ka}(t)$

$sh(t+1) = (\alpha_{sh}*\alpha_W)*(W'_k.sh'_{ka}(t)) + U'_k*x_{ka}(t)$  -> Floating point

$sh(t+1) = ReLU(h(t+1))$

$sh(t+1) = ModReLU(h(t+1),sb), with $sb = b/(2\alpha_U)$ => Could be quantized back to $\alpha_{sh}.sh'_{ka}(t+1)$ (division by $\alpha_sh$)

# Removing the  floating point multiplication  $(\alpha_{sh}*\alpha_W)$
set an upper bound for quantization: choose $\alpha_h$ such as 
$(\alpha_{sh}*\alpha_W) = \alpha_h\alpha_W/(2\alpha_U)= 2^{kal}$


 $U'_k*x_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $W'_k.sh'_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $2^{kal}W'_k.sh'_{ka}(t)$ in $Q_{k+ka+kal-1,k+ka-2}$

$sh(t+1) \in Q_{k+ka+kal-1,k+ka-2}$ 
ReLU or ModReLU -> **BUT** division by $\alpha_{sh}=2^{kal}/\alpha_W$ before quantizing  to $Q_{ka}$

# Removing the  floating point division  $\alpha_{sh}$ for quantization
require to set  $\alpha_{sh}=2^{kal}$
quantize $\alpha_W \in Q_{kw,kw'}$


 $U'_k*x_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $W'_k.sh'_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $\alpha_W.W'_k.sh'_{ka}(t)$ in $Q_{k+ka+kw-2,k+ka+kw'-2}$

$sh(t+1) \in Q_{k+ka+kw-2,k+ka+kw'-2}$ 
ReLU or ModReLU ->  shift by $\alpha_{sh}=2^{kal}$ before quantizing  to $Q_{ka}$